In [9]:
## Creating clean templates to use it for creating the new templates for each code spreadsheet
import xlwings as xw
import pandas as pd



##########################################################################################################################
#*************************************** CREATING EMPTY TEMPLATES FOR IPCA AND CDI **************************************#
##########################################################################################################################

## Grabbing workbooks
wb_template = xw.Book('template/template.xlsx')
wb_target = xw.Book('links_test.xlsx')


## Grabbing worksheets
ws_template_ipca = wb_template.sheets('Template_IPCA')
ws_template_cdi = wb_template.sheets('Template_CDI')
## Trying to add the template worksheet, if it already exists, we simply ignore it and just grab the existing worksheet
try:
    wb_target.sheets.add('Template_Limpo_IPCA')
    wb_target.sheets.add('Template_Limpo_CDI')
    ws_copy_template_ipca = wb_target.sheets('Template_Limpo_IPCA')
    ws_copy_template_cdi = wb_target.sheets('Template_Limpo_CDI')
except:
    ws_copy_template_ipca = wb_target.sheets('Template_Limpo_IPCA')
    ws_copy_template_cdi = wb_target.sheets('Template_Limpo_CDI')


## FOR PANDAS TO READ FROM
## This is the "links" worksheet
ws_links = wb_target.sheets['Sheet1']
xlsx = pd.ExcelFile('links_test.xlsx')
## Reading the worksheet so we can get the data out of it First inserting the file name, and the second argument is the worksheet
df = pd.read_excel(xlsx, ws_links.name)

## Gabbring all the elements from the CETIP list (the result is a Series/Column) / Converting the column into an array with the CETIP values
cetip_list_values = df['CETIP'].values
## Getting the first CETIP row code for test
row_1 = df.loc[df['CETIP'] == cetip_list_values[0]]



## Copying one template to another
ws_template_ipca.range('A1:AA15').copy(ws_copy_template_ipca.range('A1:AA15'))
ws_template_cdi.range('A1:AA15').copy(ws_copy_template_cdi.range('A1:AA15'))

## Deleting all the useless data inside this spreadsheet, to make a clean one
ws_full_copied_ipca = wb_target.sheets('Template_Limpo_IPCA')
ws_full_copied_cdi = wb_target.sheets('Template_Limpo_CDI')
## Deleting all useless data from the rest of the spreadsheet
ws_full_copied_ipca.range('B2:AA15').delete()
ws_full_copied_cdi.range('B2:AA15').delete()
## Deleteing useless values from the first table on top left corner
ws_full_copied_ipca.range('B2:B7').value = 0
ws_full_copied_cdi.range('B2:B8').value = 0

## Saving the changes
wb_target.save('clean_links_with_template.xlsx')


In [ ]:
##########################################################################################################################
#********************************** FILLING UP A TEMPLATE BASED ON INDEX (IPCA OR CDI) **********************************#
##########################################################################################################################

### CURRENTLY WORKING VERSION
import requests
import pandas as pd
from pandas.tseries.offsets import MonthEnd



#*************************************************** GRABBING FILES *****************************************************#
## Grabbing workbook with links and template
wb = xw.Book('clean_links_with_template.xlsx')

## Importing csv with download link
## This is the "links" worksheet
ws_links = wb.sheets['Sheet1']
xlsx = pd.ExcelFile('links_test.xlsx')
## Reading the worksheet so we can get the data out of it First inserting the file name, and the second argument is the worksheet
df_with_links = pd.read_excel(xlsx, ws_links.name)


#**************************************** GRABBING LINKS FOR EACH "FIDUCIARIA" ******************************************#  
## Grabbing only Oliveira Trust Links
oliveira_links = df_with_links.loc[df_with_links['Fiduciaria'] == 'Oliveira Trust']
print(len(oliveira_links))
## Grabbing only Vortx Links
vortx_links = df_with_links.loc[df_with_links['Fiduciaria'] == 'Vortx']
print(len(vortx_links))

## Declaring the dataframe where things will get appended to
## Defining a list to store all the data frames that will get concatenated together
list_of_df = []
## Defining a list to append the codes that, for some reason, still have no data to display
codes_with_no_data = []

##########################################################################################################################
##########################################################################################################################
#****************************************** BEGGINING OF OLIVEIRA TRUST LOOP ********************************************#
##########################################################################################################################
##########################################################################################################################
for link in oliveira_links['Link']:
    
    ## Setting the link to grab the data
    url = link
    ## Printing the url just to check if its what i expect to be
    print(url)

    ## This is for treating codes that still display no data / could not be fetched
    try:
        ##########################################################################################################################
        #********************************************** TREATING OLIVEIRA TRUST DATA ********************************************#
        ##########################################################################################################################

        ## Reading the HTML and formatting it into a dataframe
        initial_df = pd.read_html(url, encoding='utf8', skiprows=1, header=0)[0]
        ## Renaming dataframe columns
        initial_df.rename(columns={'Juros.1':'Pagamento de Juros', 'Premio.1':'Pagamento de Premio'}, inplace=True)
        
        ## Converting "Data" to datetime format
        initial_df['Data'] = pd.to_datetime(initial_df['Data']).dt.date

        ## Defining new dataframe with only the columns i'm going to use
        new_df = initial_df[['Data', 'Valor Nominal', 'Juros', 'Pagamento de Juros', 'Amortização', 'Total', 'P.U.']]

        ## Creating a new Dataframe with only Month End data
        new_df['Data'] = pd.to_datetime(new_df['Data']) + MonthEnd(1)

        ## Creating a list with the values to concat later (as concat only accepts lists)
        ## Declaring the two vectors we need to append this data (and also so they reset on the next code in loop)
        ## CETIP codes
        cetip = []
        ## "FIDUCIARIA" name
        fiduciaria = []

        ## Grabbing the row with the code to this specific Likn/URL on the loop
        row = oliveira_links.loc[oliveira_links['Link'] == link]

        ## Looping to the number of elements in the dataframe so we match the dataframe to append
        for element in new_df['Data']:
            cetip.append(row['CETIP'].iat[0])
            fiduciaria.append(row['Fiduciaria'].iat[0])

        ## Adding these "CETIP" AND "FIDUCIARIA" columns to the dataframe
        new_df.insert(0, 'Fiduciaria', fiduciaria)
        new_df.insert(0, 'CETIP', cetip)

        ## Sorting by oldest date
        new_df.sort_values(by='Data', inplace=True)

        ##########################################################################################################################
        #**************************** CREATING NEW SPREADSHEET FOR THE CODE AND FILLING IT UP ***********************************#
        ##########################################################################################################################
        
        ## Grabbing the code's index
        code_index = row['Indexador'].iat[0]

        ## Grabbing the CRI's name
        cri_name = row['Nome CRI'].iat[0]

        ## Creating vector index
        index_vector = ['IPCA', 'CDI', 'IGPM']

        ## Checking which code index it is, so we can define which template to use
        ############ CDI ###############
        if code_index in index_vector[1]:
            ## Trying to add the template worksheet, if it already exists, 
            # we simply ignore it and just grab the existing worksheet
            try:
                wb.sheets.add(cri_name)
                ws_code_cdi = wb.sheets(cri_name)
            ## If the creation of the spreadsheet fails, 
            # it indicates that it already exists, so we just pass
            except:
                pass
        ########### IPCA ETC #############
        else:
            ## Trying to add the template worksheet, if it already exists, 
            # we simply ignore it and just grab the existing worksheet
            try:
                wb.sheets.add(cri_name)
                ws_code_ipca = wb.sheets(cri_name)
            ## If the creation of the spreadsheet fails, 
            # it indicates that it already exists, so we just pass
            except:
                pass
        

        ### OLD PIECE OF CODE
        # ## Appending each formatted dataframe to the list of dataframes
        # list_of_df.append(new_df)

    ## This is for treating codes that still display no data / could not be fetched
    except:
        row = vortx_links.loc[vortx_links['Link'] == link]
        codes_with_no_data.append(row['CETIP'].iat[0])

##########################################################################################################################
##########################################################################################################################
#********************************************* END OF OLIVEIRA TRUST LOOP ***********************************************#
##########################################################################################################################
##########################################################################################################################


##########################################################################################################################
##########################################################################################################################
#******************************************** BEGGINING OF VORTX TRUST LOOP *********************************************#
##########################################################################################################################
##########################################################################################################################
for link in vortx_links['Link']:
    
    ## Setting the link to grab the data
    url = link
    ## Printing the url just to check if its what i expect to be
    print(url)

    ## This is for treating codes that still display no data / could not be fetched
    try:
        ##########################################################################################################################
        #********************************************** FORMATTING THE JSON DATA ************************************************#
        ##########################################################################################################################
        ## Getting the page with the data
        response = requests.request("GET", url)
        ## Transforming it into a raw json
        raw_json_txt = response.json()
        ## Normalizing the json data into the format we need
        json_txt = pd.json_normalize(raw_json_txt, "unitPrices", ["Data", "operationId", "smartBondId", "Pagamento de Juros", "Amortização", \
            "Pagamento Total", "PU(completo)", "PU(vazio)", "Juros", "Valor Nominal"], errors='ignore',record_prefix='_')

        ############################################################################################################################
        ############################################################################################################################

        ## Converting into Dataframe
        df = pd.DataFrame(json_txt)

        ## Renaming Columns
        df = df.rename(columns= {'_paymentDate':'Data','_interest':'Pagamento de Juros','_amortization':'Amortização','_total':'Total','_unitPriceFull':'P.U.','_interestValue':'Juros','_nominalValue':'Valor Nominal'})

        ############################################################################################################################
        ############################################################################################################################

        
        ## Getting only the columns we need
        new_df = df[['Data', 'Valor Nominal', 'Juros','Pagamento de Juros', 'Amortização', 'Total', 'P.U.']]
        


        ## Removing duplicated column names
        new_df = new_df.loc[:,~new_df.columns.duplicated()]

        ############################################################################################################################
        ############################################################################################################################
        ## Creating a new Dataframe with only Month End data
        new_df['Data'] = pd.to_datetime(new_df['Data']) + MonthEnd(1)

        ## Creating a list with the values to concat later (as concat only accepts lists)
        ## Declaring the two vectors we need to append this data (and also so they reset on the next code in loop)
        ## CETIP codes
        cetip = []
        ## "FIDUCIARIA" name
        fiduciaria = []

        ## Grabbing the row with the code to this specific Likn/URL on the loop
        row = vortx_links.loc[vortx_links['Link'] == link]

        ## Looping to the number of elements in the dataframe so we match the dataframe to append
        for element in new_df['Data']:
            cetip.append(row['CETIP'].iat[0])
            fiduciaria.append(row['Fiduciaria'].iat[0])

        ## Adding these "CETIP" AND "FIDUCIARIA" columns to the dataframe
        new_df.insert(0, 'Fiduciaria', fiduciaria)
        new_df.insert(0, 'CETIP', cetip)

        ############################################################################################################################
        ############################################################################################################################

        ## Sorting by oldest date
        new_df.sort_values(by='Data')

        ## Appending each formatted dataframe to the list of dataframes
        list_of_df.append(new_df)

    ## This is for treating codes that still display no data / could not be fetched
    except:
        row = vortx_links.loc[vortx_links['Link'] == link]
        codes_with_no_data.append(row['CETIP'].iat[0])

##########################################################################################################################
##########################################################################################################################
#*********************************************** END OF VORTX TRUST LOOP ************************************************#
##########################################################################################################################
##########################################################################################################################


##########################################################################################################################
##########################################################################################################################
#****************************************** PUTTING ALL DATAFRAMES TOGETHER *********************************************#
##########################################################################################################################
##########################################################################################################################


## Putting together all the dataframes we have
concatenated_list_of_df = pd.concat(list_of_df)

## Converting date to only year and month
concatenated_list_of_df['Data'] = pd.to_datetime(concatenated_list_of_df['Data'], format='%Y-%m-%d')
concatenated_list_of_df['Mes e Ano'] =  concatenated_list_of_df['Data'].dt.to_period('M')

## Summing for last day of each month
#final_dataframe = concatenated_list_of_df.groupby(['Fiduciaria', 'CETIP','Mes e Ano'])[['Amortização', 'Pagamento de Juros', 'Total']].sum().reset_index()
sum_last_day_of_month_values = concatenated_list_of_df.groupby(['Fiduciaria', 'CETIP', 'Data']).agg({'Valor Nominal':'min' ,'Amortização':'sum', 'Pagamento de Juros':'sum', 'Total':'sum'}).reset_index()

#[['Pagamento de Juros']].sum().reset_index()

## Removing duplicate date values, to only get the last date (day) of each row
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
# sum_last_day_of_month_values = sum_last_day_of_month_values.drop_duplicates(subset = ['Data'], keep ='last').reset_index(drop = True)
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
## Transforming the datetime with only Year/Month/Day (no hour display) ## ADDING STRFTIME!!!
sum_last_day_of_month_values['Data'] = sum_last_day_of_month_values['Data'].dt.strftime('%d/%m/%Y')
#$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

## Generating an excel from it, so we can check the results
sum_last_day_of_month_values.to_excel('organized_amortization_data.xlsx', encoding='utf8', index=False)

## Printing the concatenated list of dataframes
sum_last_day_of_month_values

#print(len(list_of_df))

#concatenated_list_of_df
#sum_last_day_of_month_values

if codes_with_no_data:
    ## Creating an excel withh all the codes that have no data/could not get fetched
    no_data_codes_df = pd.DataFrame(codes_with_no_data, columns=['CETIP SEM DADOS'])
    no_data_codes_df.to_excel('no_data_codes.xlsx', encoding='utf8', index=False)
    print(codes_with_no_data)


In [4]:
############################ TESTING CREATION OF SPREADSHEETS ####################################

### CURRENTLY WORKING VERSION
import requests
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import xlwings as xw
## Used to convert float with points correctly
from locale import atof, setlocale, LC_NUMERIC
## Setting the locale to 'pt_BR.UTF-8', as it has a number punctuation as we would like to use
setlocale(LC_NUMERIC, 'pt_BR.UTF-8')#'French_Canada.1252')



#*************************************************** GRABBING FILES *****************************************************#
## Grabbing workbook with links and template
wb = xw.Book('clean_links_with_template.xlsx')

## Importing csv with download link
## This is the "links" worksheet
ws_links = wb.sheets['Sheet1']
xlsx = pd.ExcelFile('links_test.xlsx')
## Reading the worksheet so we can get the data out of it First inserting the file name, and the second argument is the worksheet
df_with_links = pd.read_excel(xlsx, ws_links.name)


#**************************************** GRABBING LINKS FOR EACH "FIDUCIARIA" ******************************************#  
## Grabbing only Oliveira Trust Links
oliveira_links = df_with_links.loc[df_with_links['Fiduciaria'] == 'Oliveira Trust']
print(len(oliveira_links))
## Grabbing only Vortx Links
vortx_links = df_with_links.loc[df_with_links['Fiduciaria'] == 'Vortx']
print(len(vortx_links))

## Declaring the dataframe where things will get appended to
## Defining a list to store all the data frames that will get concatenated together
list_of_df = []
## Defining a list to append the codes that, for some reason, still have no data to display
codes_with_no_data = []

##########################################################################################################################
##########################################################################################################################
#****************************************** BEGGINING OF OLIVEIRA TRUST LOOP ********************************************#
##########################################################################################################################
##########################################################################################################################
for link in oliveira_links['Link']:
    
    ## Setting the link to grab the data
    url = link
    ## Printing the url just to check if its what i expect to be
    print(url)

    ## This is for treating codes that still display no data / could not be fetched
    try:
        ##########################################################################################################################
        #********************************************** TREATING OLIVEIRA TRUST DATA ********************************************#
        ##########################################################################################################################

        ## Reading the HTML and formatting it into a dataframe
        initial_df = pd.read_html(url, encoding='utf8', skiprows=1, header=0)[0]
        ## Renaming dataframe columns
        initial_df.rename(columns={'Juros.1':'Pagamento de Juros', 'Premio.1':'Pagamento de Premio'}, inplace=True)
        
        ## Converting "Data" to datetime format
        initial_df['Data'] = pd.to_datetime(initial_df['Data']).dt.date

        ## Defining new dataframe with only the columns i'm going to use
        new_df = initial_df[['Data', 'Valor Nominal', 'Juros', 'Pagamento de Juros', 'Amortização', 'Total', 'P.U.']]

        ## Creating a new Dataframe with only Month End data
        new_df['Data'] = pd.to_datetime(new_df['Data']) + MonthEnd(1)

        ## Creating a list with the values to concat later (as concat only accepts lists)
        ## Declaring the two vectors we need to append this data (and also so they reset on the next code in loop)
        ## CETIP codes
        cetip = []
        ## "FIDUCIARIA" name
        fiduciaria = []

        ## Grabbing the row with the code to this specific Likn/URL on the loop
        row = oliveira_links.loc[oliveira_links['Link'] == link]
        print(row)
        
        ## Looping to the number of elements in the dataframe so we match the dataframe to append
        for element in new_df['Data']:
            cetip.append(row['CETIP'].iat[0])
            fiduciaria.append(row['Fiduciaria'].iat[0])

        ## Adding these "CETIP" AND "FIDUCIARIA" columns to the dataframe
        new_df.insert(0, 'Fiduciaria', fiduciaria)
        new_df.insert(0, 'CETIP', cetip)

        ## Sorting by oldest date
        new_df.sort_values(by='Data', inplace=True)

        ##########################################################################################################################
        #**************************** CREATING NEW SPREADSHEET FOR THE CODE AND FILLING IT UP ***********************************#
        ##########################################################################################################################
        
        #*********************************************** INFO FOR TOP LEFT CORNER ***********************************************#
        ## Creating vector index
        index_vector = ['IPCA', 'CDI', 'IGPM']


        #******************* Treating the tax string so we retrieve only what we want *******************#
        ## Grabbing the tax (TAXA)
        tax_string_full = row['Remuneração'].iat[0]

        ## Initializing index percentage as '?' so we can check it if exists, if it does, we use it, if not, we change it to 100%
        index_percentage = '?'
        # Splitting the string on the '+' sign if it cointains it
        # On index 0, will be everything up to '+', and on index 1 will be the rest after '+'
        if '+' in tax_string_full:
            split_tax_string = tax_string_full.split('+')
            ## Treating the case where we have the index percentage before de '+' sign
            if '%' in split_tax_string[0]:
                ## Grabbing the string that contains de % sign
                index_string_with_percentage = split_tax_string[0]
                ## Treating the string so it only has the number with the % sign
                ## Converting it to float and rouding it to 2 decimals
                index_percentage = round(float((index_string_with_percentage[:index_string_with_percentage.rfind('%')]).replace(',', '.')), 2)
                ## Adding a % sign ate the end
                index_percentage = str(index_percentage) + '%'

            ## Grabbing the string that contains de % sign
            tax_string_with_percentage = split_tax_string[1]
            ## Treating the string so it only has the number with the % sign
            ## Converting it to float and rouding it to 2 decimals
            tax_percentage = round(float((tax_string_with_percentage[:tax_string_with_percentage.rfind('%')]).replace(',', '.')), 2)
            
            tax_percentage = str(tax_percentage) + '%'
        else:
            ## Treating the string so it only has the number with the % sign
            ## Converting it to float and rouding it to 2 decimals
            tax_percentage = round(float((tax_string_full[:tax_string_full.rfind('%')]).replace(',', '.')), 2)
            ## Adding a % sign ate the end
            tax_percentage = str(tax_percentage) + '%'

        ## WE WILL BE USING THE VARIABLES --> tax_percentage (as TAXA) and index_percentage (% DA TAXA)

    
        ## Grabbing the code's index (INDEXADOR)
        code_index = row['Indexador'].iat[0]

        ## Defining month delay (DELAY (MESES))
        delay = 1

        ## Defining the NOMINAL VALUE (VOLUME (VALOR NOMINAL))
        nominal_value = row['P.U. Inicial (Valor Nominal)'].iat[0]

        ## Defining the CETIP code (CÓDIGO)
        cetip_code = row['CETIP'].iat[0]

        ## Defining the number of units (NÚMERO DE COTAS) - Converting it to float
        num_of_units = atof(row['Quantidade de Ativos'].iat[0])

        ## Defining emission date
        emission_date = row['Data Emissão'].iat[0]

        ## Defining the two vectors we will be using for each template
        ## IPCA VECTOR
        top_left_corner_ipca_vector = [tax_percentage, code_index, delay, nominal_value, cetip_code, num_of_units]
        ## Checking if the index_percentage was given, if it was not given, we consider it 100%
        ## CDI VECTOR
        if index_percentage == '?':
            index_percentage = '100%'
            top_left_corner_cdi_vector = [tax_percentage, code_index, index_percentage, nominal_value, cetip_code, num_of_units, emission_date]
        else:
            top_left_corner_cdi_vector = [tax_percentage, code_index, index_percentage, nominal_value, cetip_code, num_of_units, emission_date]

        #########################################################################################################################
        #******************************* UP TO HERE WE'VE GRABBED ALL THE INFO FOR TOP LEFT CORNER *****************************#
        #########################################################################################################################

        ## Grabbing the CRI's name
        cri_name = row['Nome CRI'].iat[0]

        print(top_left_corner_ipca_vector)
        print(top_left_corner_cdi_vector)
        #########################################################################################################################
        #********************************************* DEFINING WHICH TEMPLATE TO USE ******************************************#
        #########################################################################################################################
        ## Checking which code index it is, so we can define which template to use
        ############ CDI ###############
        if code_index in index_vector[1]:
            ## Trying to add the template worksheet, if it already exists, 
            # we simply ignore it and just grab the existing worksheet
            try:
                print(cri_name)
                ## Trying to create the worksheet
                wb.sheets.add(cri_name)
                ##  Grabbing the newly created worksheet
                ws_code_cdi_new = wb.sheets(cri_name)
                ## Grabbing the empty template
                ws_clean_cdi_template = wb.sheets('Template_Limpo_CDI')

                #**************************************** COPYING THE TEMPLATE ***************************************# 
                ws_clean_cdi_template.range('A1:AA15').copy(ws_code_cdi_new.range('A1:AA15'))

                #************************************* FILLING UP TOP LEFT CORNER ************************************#
                ## Filling up each row with the data we've gathered for CDI template
                for i in range(0, len(top_left_corner_cdi_vector)):
                    #print(cri_name)
                    #print(i)
                    #print(top_left_corner_cdi_vector[i])
                    ws_clean_cdi_template.range('B' + str(i+2)).value = top_left_corner_cdi_vector[i]

            ## If the creation of the spreadsheet fails, 
            # it indicates that it already exists, so we just pass
            except:
                # #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
                # ##  Grabbing the newly created worksheet
                # ws_code_cdi_new = wb.sheets(cri_name)
                # ## Grabbing the empty template
                # ws_clean_cdi_template = wb.sheets('Template_Limpo_CDI')

                # #**************************************** COPYING THE TEMPLATE ***************************************# 
                # ws_clean_cdi_template.range('A1:AA15').copy(ws_code_cdi_new.range('A1:AA15'))

                # #************************************* FILLING UP TOP LEFT CORNER ************************************#
                # ## Filling up each row with the data we've gathered for CDI template
                # for i in range(0, len(top_left_corner_cdi_vector)):
                #     ws_clean_cdi_template.range('B' + str(i+2)).value = top_left_corner_cdi_vector[i]
                pass
                #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
        ########### IPCA ETC #############
        else:
            ## Trying to add the template worksheet, if it already exists, 
            # we simply ignore it and just grab the existing worksheet
            try:
                print(cri_name)
                ## Trying to create the worksheet
                wb.sheets.add(cri_name)
                ##  Grabbing the newly created worksheet
                ws_code_ipca_new = wb.sheets(cri_name)
                ## Grabbing the empty template
                ws_clean_ipca_template = wb.sheets('Template_Limpo_IPCA')

                #**************************************** COPYING THE TEMPLATE ***************************************# 
                ws_clean_ipca_template.range('A1:AA15').copy(ws_code_ipca_new.range('A1:AA15'))

                print(ws_code_ipca_new.name)
                #************************************* FILLING UP TOP LEFT CORNER ************************************#
                ## Filling up each row with the data we've gathered for IPCA and others template
                for i in range(0, len(top_left_corner_ipca_vector)):
                    #print(cri_name)
                    #print(i)
                    #print(top_left_corner_ipca_vector[i])
                    ws_clean_ipca_template.range('B' + str(i+2)).value = top_left_corner_ipca_vector[i]

            ## If the creation of the spreadsheet fails, 
            # it indicates that it already exists, so we just pass
            except:
                # #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
                # ##  Grabbing the newly created worksheet
                # ws_code_ipca_new = wb.sheets(cri_name)
                # ## Grabbing the empty template
                # ws_clean_ipca_template = wb.sheets('Template_Limpo_IPCA')

                # #**************************************** COPYING THE TEMPLATE ***************************************# 
                # ws_clean_ipca_template.range('A1:AA15').copy(ws_code_ipca_new.range('A1:AA15'))

                # #************************************* FILLING UP TOP LEFT CORNER ************************************#
                # ## Filling up each row with the data we've gathered for IPCA and others template
                # for i in range(0, len(top_left_corner_ipca_vector)):
                #     ws_clean_ipca_template.range('B' + str(i+2)).value = top_left_corner_ipca_vector[i]
                # #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
                pass
    except:
        row = vortx_links.loc[vortx_links['Link'] == link]
        codes_with_no_data.append(row['CETIP'].iat[0])

27
11
https://www.oliveiratrust.com.br/fiduciario/pu_hist.php?ativo=cri&tit=21421&excel=1


C:\Users\clib_\AppData\Local\Temp\ipykernel_24824\2868367734.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Data'] = pd.to_datetime(new_df['Data']) + MonthEnd(1)
c:\Users\clib_\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


    Nome CRI       CETIP                                               Link  \
0  AL Cambuí  20J0763944  https://www.oliveiratrust.com.br/fiduciario/pu...   

       Fiduciaria Indexador                   Remuneração  \
0  Oliveira Trust      IPCA  8,50000000% a.a. na base 252   

  Quantidade de Ativos Data Emissão Data Vencimento  \
0                9.750   23/10/2020      30/10/2025   

  P.U. Inicial (Valor Nominal)  
0                     1.000,00  
['8.5%', 'IPCA', 1, '1.000,00', '20J0763944', 9750.0]
['8.5%', 'IPCA', '100%', '1.000,00', '20J0763944', 9750.0, '23/10/2020']
AL Cambuí
AL Cambuí
AL Cambuí
0
8.5%
AL Cambuí
1
IPCA
AL Cambuí
2
1
AL Cambuí
3
1.000,00
AL Cambuí
4
20J0763944
AL Cambuí
5
9750.0
https://www.oliveiratrust.com.br/fiduciario/pu_hist.php?ativo=cri&tit=21331&excel=1


C:\Users\clib_\AppData\Local\Temp\ipykernel_24824\2868367734.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Data'] = pd.to_datetime(new_df['Data']) + MonthEnd(1)
c:\Users\clib_\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


    Nome CRI       CETIP                                               Link  \
1  AMF Saúde  20J0459371  https://www.oliveiratrust.com.br/fiduciario/pu...   

       Fiduciaria Indexador                   Remuneração  \
1  Oliveira Trust      IPCA  8,00000000% a.a. na base 252   

  Quantidade de Ativos Data Emissão Data Vencimento  \
1               17.600   08/10/2020      30/10/2025   

  P.U. Inicial (Valor Nominal)  
1                     1.000,00  
['8.0%', 'IPCA', 1, '1.000,00', '20J0459371', 17600.0]
['8.0%', 'IPCA', '100%', '1.000,00', '20J0459371', 17600.0, '08/10/2020']
AMF Saúde
AMF Saúde
AMF Saúde
0
8.0%
AMF Saúde
1
IPCA
AMF Saúde
2
1
AMF Saúde
3
1.000,00
AMF Saúde
4
20J0459371
AMF Saúde
5
17600.0
https://www.oliveiratrust.com.br/fiduciario/pu_hist.php?ativo=cri&tit=21301&excel=1


...

        Nome CRI       CETIP  \
24  Tecnisa 344S  20L0675397   

                                                 Link      Fiduciaria  \
24  https://www.oliveiratrust.com.br/fiduciario/pu...  Oliveira Trust   

   Indexador                          Remuneração Quantidade de Ativos  \
24      IPCA  IPCA + 6,85000000% a.a. na base 252               50.000   

   Data Emissão Data Vencimento P.U. Inicial (Valor Nominal)  
24   22/12/2020      30/12/2025                     1.000,00  
['6.85%', 'IPCA', 1, '1.000,00', '20L0675397', 50000.0]
['6.85%', 'IPCA', '100%', '1.000,00', '20L0675397', 50000.0, '22/12/2020']
Tecnisa 344S
Tecnisa 344S
Tecnisa 344S
0
6.85%
Tecnisa 344S
1
IPCA
Tecnisa 344S
2
1
Tecnisa 344S
3
1.000,00
Tecnisa 344S
4
20L0675397
Tecnisa 344S
5
50000.0
https://www.oliveiratrust.com.br/fiduciario/pu_hist.php?ativo=cri&tit=24101&excel=1


C:\Users\clib_\AppData\Local\Temp\ipykernel_24824\2868367734.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Data'] = pd.to_datetime(new_df['Data']) + MonthEnd(1)
c:\Users\clib_\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


        Nome CRI       CETIP  \
25  Tecnisa 397S  21G0568394   

                                                 Link      Fiduciaria  \
25  https://www.oliveiratrust.com.br/fiduciario/pu...  Oliveira Trust   

   Indexador                          Remuneração Quantidade de Ativos  \
25      IPCA  IPCA + 7,00000000% a.a. na base 252              100.000   

   Data Emissão Data Vencimento P.U. Inicial (Valor Nominal)  
25   15/07/2021      29/06/2028                     1.000,00  
['7.0%', 'IPCA', 1, '1.000,00', '21G0568394', 100000.0]
['7.0%', 'IPCA', '100%', '1.000,00', '21G0568394', 100000.0, '15/07/2021']
Tecnisa 397S
Tecnisa 397S
Tecnisa 397S
0
7.0%
Tecnisa 397S
1
IPCA
Tecnisa 397S
2
1
Tecnisa 397S
3
1.000,00
Tecnisa 397S
4
21G0568394
Tecnisa 397S
5
100000.0
https://www.oliveiratrust.com.br/fiduciario/pu_hist.php?ativo=cri&tit=21201&excel=1


C:\Users\clib_\AppData\Local\Temp\ipykernel_24824\2868367734.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Data'] = pd.to_datetime(new_df['Data']) + MonthEnd(1)
c:\Users\clib_\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


          Nome CRI       CETIP  \
26  Tenente Negrão  20I0696446   

                                                 Link      Fiduciaria  \
26  https://www.oliveiratrust.com.br/fiduciario/pu...  Oliveira Trust   

   Indexador                           Remuneração Quantidade de Ativos  \
26      IPCA  IPCA + 10,00000000% a.a. na base 252               13.300   

   Data Emissão Data Vencimento P.U. Inicial (Valor Nominal)  
26   22/09/2020      29/09/2025                     1.000,00  
['10.0%', 'IPCA', 1, '1.000,00', '20I0696446', 13300.0]
['10.0%', 'IPCA', '100%', '1.000,00', '20I0696446', 13300.0, '22/09/2020']
Tenente Negrão
Tenente Negrão
Tenente Negrão
0
10.0%
Tenente Negrão
1
IPCA
Tenente Negrão
2
1
Tenente Negrão
3
1.000,00
Tenente Negrão
4
20I0696446
Tenente Negrão
5
13300.0


In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import pandas as pd
import time
import xlwings as xw

## Defining chrome driver path
chrome_driver_path = 'DRIVER/chromedriver.exe'

##########################################################################################################################
##########################################################################################################################
#********************************************** Creating the list of codes **********************************************#
##########################################################################################################################
##########################################################################################################################
# ## Opening the file in read mode - ACTUAL FILE
# my_file = open("CODIGOS/codigos_resumido.txt", "r")
# # Reading the file
# data = my_file.read()
# # Replacing End: splitting the text when newline ('\n') is seen.
# list_of_codes = data.split("\n")
# #print(list_of_codes)
# my_file.close()

wb_template = xw.Book('template/template.xlsx')

ws = wb_template.sheets('Resumo')

list_of_cri_names = ws.range('A4:A43').value
list_of_codes = ws.range('B4:B43').value

print(list_of_cri_names)

['AL Cambuí', 'AMF Saúde', 'AR Terrenos', 'Carinás', 'Eusébio Matoso', 'Fashion Mall 339S', 'Fashion Mall 340S', 'Francisco Leitão 14S', 'Francisco Leitão 15S', 'Gafisa 27S', 'Gafisa 28S', 'General Shopping', 'GPA 79S', 'GPA 83S', 'GTLG', 'Helbor 113S', 'Helbor 440S', 'HGLG Eletrolux', 'Inter 464S', 'JSL Ribeira 261S', 'JSL Ribeira 268S', 'Mabu 204S', 'Manhattan 196S', 'Monvert', 'OKA', 'Oscar Freire 50S', 'PG Rodrigues Alves', 'Planta 47S', 'Praias Canoas', 'Raposo Shopping', 'Rosewood', 'São Gonçalo', 'SER Educacional 284S', 'Socicam', 'Tarjab Altino Arantes', 'Tecnisa 175S', 'Tecnisa 344S', 'Tecnisa 397S', 'Tenente Negrão', 'Villa XP']
